# Instalação das libs 
### Com exceção do spaCy, todas as libs já estão instaladas no ambiente anaconda. Caso seja executado em outro ambiente, será necessário instalar cada uma delas através do pip.

In [ ]:
!pip install -q spacy==2.2.3

# Importação das libs

In [ ]:
import spacy
import pandas as pd
import string
import spacy
import random
import seaborn as sns
import numpy as np
import re
from sklearn.metrics import confusion_matrix, accuracy_score

spacy.__version__

# Download do idioma em português do spaCy

In [ ]:
!python3 -m spacy download pt

# Importação do dataset de Treinamento

In [ ]:
base_treinamento = pd.read_csv('/content/Train3Classes.csv', delimiter=';')

In [ ]:
# Listagem dos 5 primeiros registros do arquivo
base_treinamento.head()

In [ ]:
# Listagem dos 5 últimos registros do arquivo
base_treinamento.tail()

In [ ]:
# Visualização da distribuição de amostras para cada label
# Aqui é possível identificar que o dataset possui a mesma quantidade para cada rótulo
sns.countplot(base_treinamento['sentiment'], label = 'Contagem');

In [ ]:
# Remoção das colunas que não serão utilizadas no projeto
base_treinamento.drop(['id', 'tweet_date', 'query_used'], axis = 1, inplace=True)

In [ ]:
# Visualização de como ficou o dataset após remoção das colunas
base_treinamento.head()

In [ ]:
# No heatmap é possível visualizar que não existe nenhum registro faltante. 
# Não há valores nulos, a base está completamente preenchida
sns.heatmap(pd.isnull(base_treinamento));

In [ ]:
# Leitura da base de Teste
base_teste = pd.read_csv('/content/Test3classes.csv', delimiter=';')

In [ ]:
# Listagem dos 5 primeiros registros da base de teste
base_teste.head()

In [ ]:
# Na base de Teste também, a distribuição de rótulos está igualmente distribuída.
sns.countplot(base_teste['sentiment'], label='Contagem');

In [ ]:
# Remoção das colunas não utilizadas
base_teste.drop(['id', 'tweet_date', 'query_used'], axis = 1, inplace=True)

In [ ]:
# A base de testes também não possui nenhum valor faltante.
sns.heatmap(pd.isnull(base_teste));

In [ ]:
# Criação do modelo vazio
pln = spacy.load('pt')
pln

In [ ]:
# Stopwords que o spacy possui para a 
stop_words = spacy.lang.pt.stop_words.STOP_WORDS
len(stop_words)
# RA palavra "não" foi removida da lista de stopwords porque o modelo apresentou melhor performance considerando-a na sentença
stop_words.remove('não')
len(stop_words)

In [ ]:
# função que faz todo o pré-processamento das sentenças.
def preprocessamento(texto):
  # Letras minúsculas
  texto = texto.lower()

  # Nome do usuário
  texto = re.sub(r"@[A-Za-z0-9$-_@.&+]+", ' ', texto)

  # URLs
  texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)

  # Espaços em branco
  texto = re.sub(r" +", ' ', texto)

  # Emoticons
  lista_emocoes = {':)': 'emocaopositiva',
                   ':>': 'emocaopositiva',
                   ':3': 'emocaopositiva',
                   ':d': 'emocaopositiva',
                   ':]': 'emocaopositiva',
                   ':}': 'emocaopositiva',
                   '=)': 'emocaopositiva',
                   '=>': 'emocaopositiva',
                   '=3': 'emocaopositiva',
                   '=]': 'emocaopositiva',
                   '=}': 'emocaopositiva',
                   '=d': 'emocaopositiva',
                   ':(': 'emocaonegativa'}
  
  # remove das frases os emoticons
  for emocao in lista_emocoes:
    texto = texto.replace(emocao, '')

  # Lematização
  documento = pln(texto)

  # cria uma nova lista para aplicar a lematização
  lista = []
  for token in documento:
    lista.append(token.lemma_)
  
  # Stopwords e pontuações
  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])
  
  return lista

In [ ]:
# Aplicação do prá-processamento na base de treinamento
base_treinamento['tweet_text'] = base_treinamento['tweet_text'].apply(preprocessamento)

In [ ]:
# Visualização de como ficou após o pré-processamento
base_treinamento.head(10)

In [ ]:
# Pré-processamento na base de teste
base_teste['tweet_text'] = base_teste['tweet_text'].apply(preprocessamento)

In [ ]:
# Visualização de como ficou após o pré-processamento
base_teste.head(10)

In [ ]:
# Espécie de one-hot-encoding versão dicionário de rótulos :D
base_dados_treinamento_final = []
for texto, emocao in zip(base_treinamento['tweet_text'], base_treinamento['sentiment']):
  if emocao == 1:
    dic = ({'POSITIVO': True, 'NEGATIVO': False, 'NEUTRO': False})
  elif emocao == 0:
    dic = ({'POSITIVO': False, 'NEGATIVO': True, 'NEUTRO': False})
  elif emocao == 2:
    dic = ({'POSITIVO': False, 'NEGATIVO': False, 'NEUTRO': True})

  base_dados_treinamento_final.append([texto, dic.copy()])

In [ ]:
# Exemplo aleatório de como ficou o dicionário de rótulos positivos
base_dados_treinamento_final[35:40]

In [ ]:
# Criação da pipeline para treinar o modelo
modelo = spacy.blank('pt')
categorias = modelo.create_pipe("textcat")

# Adição dos rótulos na pipeline
categorias.add_label("POSITIVO")
categorias.add_label("NEGATIVO")
categorias.add_label("NEUTRO")
modelo.add_pipe(categorias)

# essa variável vai servir para pegar o histórico de losses
historico = []

In [ ]:
# Treinamento do modelo em si :)
modelo.begin_training()

# São 20 épocas
for epoca in range(20):
  print("Epoca: ", epoca)
  random.shuffle(base_dados_treinamento_final)
  losses = {}
  # a base de treinamento foi quebrada em 256 batches
  for batch in spacy.util.minibatch(base_dados_treinamento_final, 256):
    textos = [modelo(texto) for texto, entities in batch]
    annotations = [{'cats': entities} for texto, entities in batch]
    modelo.update(textos, annotations, losses=losses)
    historico.append(losses)
  if epoca % 5 == 0:
    print(losses)

In [ ]:
historico_loss = []
for i in historico:
  historico_loss.append(i.get('textcat'))

In [ ]:
# criação de um numpy array para o histórico de erros
historico_loss = np.array(historico_loss)
historico_loss

In [ ]:
# usando matplotlib para plotar o gráfico da progressão do erro
import matplotlib.pyplot as plt
plt.plot(historico_loss)
plt.title('Progressão do erro')
plt.xlabel('Batches')
plt.ylabel('Erro')

In [ ]:
# este trecho salva o modelo (estado/pickle) treinado
modelo.to_disk("modelo")

In [ ]:
# Carrega o modelo treinado
modelo_carregado = spacy.load('modelo')
modelo_carregado

In [ ]:
# Teste com frase positiva
texto_positivo = 'eu gosto muito de você'
texto_positivo = preprocessamento(texto_positivo)
texto_positivo

In [ ]:
modelo_carregado(texto_positivo).cats

In [ ]:
# Teste com frase negativa
texto_negativo = 'não gostei nada'
texto_negativo = preprocessamento(texto_negativo)
texto_negativo

In [ ]:
modelo_carregado(texto_negativo).cats

In [ ]:
# Carrega todas as previsões da base de treinamento em uma lista para comparar com o rótulo original.
previsoes = []
for texto in base_treinamento['tweet_text']:
  previsao = modelo_carregado(texto)
  previsoes.append(previsao.cats)

In [ ]:
previsoes

In [ ]:
# Obtém uma lista com os rótulos para transformar em numpy array - para a matriz de confusão e acurácia.
previsoes_final = []
for previsao in previsoes:
  # obtém qual foi o rótulo que apresentou maior score e dá um append no número que o representa
  # 0 - Negativo, 1 - Positivo, 2 - Neutro
  valor_max = max(previsao, key = previsao.get)
  if valor_max == 'POSITIVO':
    previsoes_final.append(1)
  elif valor_max == 'NEGATIVO':
    previsoes_final.append(0)
  else:
    previsoes_final.append(2)

# gera o numpy array
previsoes_final = np.array(previsoes_final)

In [ ]:
# gera um array aqui com os rótulos do dataset de treinamento
# para isso, pega somente a coluna 'sentiment'
respostas_reais = base_treinamento['sentiment'].values

In [ ]:
# Retorna a acurácia para a base de treinamento
accuracy_score(respostas_reais, previsoes_final)

In [ ]:
# Matriz de confusão - base de treinamento
cm = confusion_matrix(respostas_reais, previsoes_final)
cm

In [ ]:
# plota a matriz de confusão
sns.heatmap(cm, annot=True)

In [ ]:
# Agora faz o mesmo com a base de teste :)
previsoes = []
for texto in base_teste['tweet_text']:
  previsao = modelo_carregado(texto)
  previsoes.append(previsao.cats)

In [ ]:
previsoes_final = []
for previsao in previsoes:
  valor_max = max(previsao, key = previsao.get)
  if valor_max == 'POSITIVO':
    previsoes_final.append(1)
  elif valor_max == 'NEGATIVO':
    previsoes_final.append(0)
  else:
    previsoes_final.append(2)

previsoes_final = np.array(previsoes_final)

In [ ]:
respostas_reais = base_teste['sentiment'].values

In [ ]:
# Acurácia da base de teste
accuracy_score(respostas_reais, previsoes_final)

In [ ]:
# Matriz de confusão da base de teste
cm = confusion_matrix(respostas_reais, previsoes_final)
cm

In [ ]:
# Plota a matriz de confusão
sns.heatmap(cm, annot=True)